In [97]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from functools import reduce
import json

In [12]:
def curr_df(year):
    r = requests.get("http://www.xe.com/currencytables/?from=USD&date={}-10-20".format(year))
    soup = BeautifulSoup(r.text, "lxml")
    table = soup.find("table", {"id": "historicalRateTbl"})
    df = pd.read_html(str(table))
    df = df[0]
    df.columns = ['curr_code', 'curr_name', 'usd_to_x', 'x_to_usd']
    return df

In [13]:
years = [i for i in range(2016, 1999, -1)]

In [14]:
dfs = []
for i in years:
    dfs.append(curr_df(i))

In [192]:
ndfs = []
start_date = 2016
for i, df in enumerate(dfs):
    new_cols = []
    for j, c in enumerate(df.columns):
        if j:
            #c = "_".join(c.split('_')[:-1])
            new_cols.append(c + "_{}".format(start_date))
        else:
            new_cols.append(c)
    df.columns = new_cols
    start_date -= 1
    ndfs.append(df)

In [234]:
bdf = reduce(lambda acc, x: pd.merge(acc, x, how='left', on='curr_code'), dfs)

In [235]:
cols = []
for i in years:
    cols.append('x_to_{}'.format(i))

In [236]:
bdf.index = dfs[0]['curr_code']

In [237]:
bdf = bdf[cols]

In [238]:
bdf.fillna(0, inplace=True)

In [127]:
currency = {}
for c in dfs[0].curr_code:
    r = requests.get("https://restcountries.eu/rest/v1/currency/%s" % c)
    if r.status_code == 200:
        j = json.loads(r.text)
        countries = []
        for i in j:
            latlng = i['latlng']
            if len(latlng) == 2:
                lat = round(latlng[0])
                lg = round(latlng[1])
                countries.append((i['name'], (lat, lg)))
        currency[c] = countries

In [239]:
bdf = bdf[bdf.index.isin(currency.keys())]

In [240]:
for c in bdf.columns:
    mn = bdf[c].min()
    mx = bdf[c].max()
    diff = mx - mn
    bdf[c] = (bdf[c] - mn)/diff

In [241]:
rec = bdf.to_records()

In [242]:
data = []
for i, y in enumerate(years):
    globe = []
    for r in rec:
        if r[0] in currency:
            for cities in currency[r[0]]:
                globe.append(cities[1][0])
                globe.append(cities[1][1])
                globe.append(r[i+1])
        
    data.append([y, globe])

In [243]:
with open('good_curr.json', 'w') as f:
    f.write(json.dumps(data))